In [1]:
import brightway2 as bw
import numpy as np
import json, os

#Local files
from import_databases import *

In [2]:
# Older databases
ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
# Newer databases
ei35_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.5 cutoff/datasets'  
# Household budget survey
habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
co_path = 'data/es8b01452_si_002.xlsx'

In [3]:
project = "consumption ei35"
bw.projects.set_current(project)

In [4]:
# Import biosphere, ecoinvent 35, ecoinvent 33
bw.bw2setup()
ei35_name = 'ecoinvent 3.5 cutoff'
import_ecoinvent(ei35_path, ei35_name)
create_ecoinvent_33_project(ei33_path)

Biosphere database already present!!! No setup is needed
ecoinvent 3.5 cutoff database already present!!! No import is needed
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed


In [5]:
# Consumption DB
co_name = CONSUMPTION_DB_NAME
if co_name in bw.databases:
    del bw.databases[co_name]
import_consumption_db(
    co_path, 
    habe_path, 
    exclude_dbs = [
        'heia', 
        'EXIOBASE 2.2',
        'Agribalyse 1.2',
        'Agribalyse 1.3 - ecoinvent 3.5 cutoff',
    ],
    ei_name=ei35_name,
)

co = bw.Database('CH consumption 1.0')
len(co)

Extracted 1 worksheets in 0.15 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.17 seconds
Applying strategy: link_iterable_by_fields
Migration for 'steam production in chemical industry' and 'market for green bell pepper'
Applying strategy: migrate_datasets
Apply

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/15/2020 17:44:01
  Finished: 10/15/2020 17:44:01
  Total time elapsed: 00:00:00
  CPU %: 106.30
  Memory %: 1.53
Created database: CH consumption 1.0


203

In [6]:
# Add average consumption as a functional unit
add_consumption_activities(co_name, habe_path)
len(co)

205

In [7]:
# LCA
co = bw.Database('CH consumption 1.0')
act = co.search('average')[0]
demand = {act: 1}
method = ('IPCC 2013', 'climate change', 'GTP 100a')
act

'ch hh average consumption' (1 month of consumption, CH, None)

In [8]:
list(act.exchanges())

[Exchange: 1 1 month of consumption 'ch hh average consumption' (1 month of consumption, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 201.46845215493406 CHF 'Other insurance premiums' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 76.54080218942872 CHF 'Fees' (CHF, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 2.6523185730240124 kilogram 'Rice' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 10.258710269091356 kilogram 'Pasta products' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 31.169177290477347 kilogram 'Bread' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumption, CH, None)>,
 Exchange: 46.03483906003796 kilogram 'Bakery products (sweet and salty)' (kilogram, CH, None) to 'ch hh average consumption' (1 month of consumptio

In [9]:
%%time
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

CPU times: user 1.78 s, sys: 178 ms, total: 1.96 s
Wall time: 778 ms


3494.756875559164